## Machine Learning in .NET with ML .NET
Because of course you can do machine learning in .NET

### Setup

This project relies on ML .NET and its Auto ML module. Both will be installed from NuGet Package Manager.

In [139]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.AutoML"

Installed Packages Microsoft.ML, 2.0.1 Microsoft.ML.AutoML, 0.20.1

In [140]:
// All the imports we'll need
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

using System.Collections.Generic;

In [141]:
// Everything in ML .NET revolves around a context object
MLContext Context = new();

### Load Datasets

By the way, this is my least favorite part of AutoML in ML.NET

In [142]:
    /// <summary>
    /// This class represents a single unreleased video game
    /// </summary>    
    public class GameInfo
    {
        [LoadColumn(0)]
        public string Title { get; set; }

        // These columns are our Features that impact the ESRB label
        [LoadColumn(1)]
        public bool Console { get; set; }
        [LoadColumn(2)]
        public bool AlcoholReference { get; set; }
        [LoadColumn(3)]
        public bool AnimatedBlood { get; set; }
        [LoadColumn(4)]
        public bool Blood { get; set; }
        [LoadColumn(5)]
        public bool BloodAndGore { get; set; }
        [LoadColumn(6)]
        public bool CartoonViolence { get; set; }
        [LoadColumn(7)]
        public bool CrudeHumor { get; set; }
        [LoadColumn(8)]
        public bool DrugReference { get; set; }
        [LoadColumn(9)]
        public bool FantasyViolence { get; set; }
        [LoadColumn(10)]
        public bool IntenseViolence { get; set; }
        [LoadColumn(11)]
        public bool Language { get; set; }
        [LoadColumn(12)]
        public bool Lyrics { get; set; }
        [LoadColumn(13)]
        public bool MatureHumor { get; set; }
        [LoadColumn(14)]
        public bool MildBlood { get; set; }
        [LoadColumn(15)]
        public bool MildCartoonViolence { get; set; }
        [LoadColumn(16)]
        public bool MildFantasyViolence { get; set; }
        [LoadColumn(17)]
        public bool MildLanguage { get; set; }
        [LoadColumn(18)]
        public bool MildLyrics { get; set; }
        [LoadColumn(19)]
        public bool MildSuggestiveThemes { get; set; }
        [LoadColumn(20)]
        public bool MildViolence { get; set; }
        [LoadColumn(21)]
        public bool NoDescriptors { get; set; }
        [LoadColumn(22)]
        public bool Nudity { get; set; }
        [LoadColumn(23)]
        public bool PartialNudity { get; set; }
        [LoadColumn(24)]
        public bool SexualContent { get; set; }
        [LoadColumn(25)]
        public bool SexualThemes { get; set; }
        [LoadColumn(26)]
        public bool SimulatedGambling { get; set; }
        [LoadColumn(27)]
        public bool StrongLanguage { get; set; }
        [LoadColumn(28)]
        public bool StrongSexualContent { get; set; }
        [LoadColumn(29)]
        public bool SuggestiveThemes { get; set; }
        [LoadColumn(30)]
        public bool UseOfAlcohol { get; set; }
        [LoadColumn(31)]
        public bool UseOfDrugsAndAlcohol { get; set; }
        [LoadColumn(32)]
        public bool Violence { get; set; }

        // This is our Label column - the one we're trying to predict
        [LoadColumn(33)]
        public string ESRBRating { get; set; }

        public override string ToString() => "{" + Title + "}";
    }

In [143]:
// Load training data. Taken from Kaggle dataset at https://www.kaggle.com/imohtn/video-games-rating-by-esrb
IDataView trainData = Context.Data.LoadFromTextFile<GameInfo>(
    path: "ESRB.csv",
    separatorChar: ',',
    hasHeader: true,
    allowQuoting: true);

In [144]:
// Load test data. Taken from Kaggle dataset at https://www.kaggle.com/imohtn/video-games-rating-by-esrb
IDataView validationData = Context.Data.LoadFromTextFile<GameInfo>(
    path: "ESRBTest.csv", 
    separatorChar: ',', 
    hasHeader: true, 
    allowQuoting: true);

### Training
This uses automated machine learning to find the best featurization and training settings for the dataset

In [145]:
uint secondsToTrain = 10;

// Configure the experiment
MulticlassClassificationExperiment experiment = 
    Context.Auto().CreateMulticlassClassificationExperiment(secondsToTrain);

In [146]:
    /// <summary>
    /// This class can report the progress of a machine learning job in progress
    /// </summary>
    public class MulticlassConsoleProgressReporter 
        : IProgress<RunDetail<MulticlassClassificationMetrics>>
    {
        public void Report(RunDetail<MulticlassClassificationMetrics> value)
        {
            if (value.ValidationMetrics != null)
            {
                double accuracy = value.ValidationMetrics.MacroAccuracy;

                Console.WriteLine($"{value.TrainerName} ran in {value.RuntimeInSeconds:0.00} seconds with accuracy of {accuracy:p}");
            }
            else
            {
                Console.WriteLine($"{value.TrainerName} ran in {value.RuntimeInSeconds:0.00} seconds but did not complete. Time likely expired.");
            }
        }
    }

In [147]:
// Synchronously Train the model
ExperimentResult<MulticlassClassificationMetrics> result = 
    experiment.Execute(
        trainData: trainData, // The data to memorize
        validationData: validationData, // How we evaluate our model's performance against new data
        labelColumnName: nameof(GameInfo.ESRBRating), // What we're trying to predict
        progressHandler: new MulticlassConsoleProgressReporter()); // How to show progress
        
// Display details about the result
result

ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 1.37 seconds with accuracy of 75.41%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown ran in 1.50 seconds with accuracy of 74.88%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown ran in 1.35 seconds with accuracy of 72.55%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown ran in 1.40 seconds with accuracy of 74.50%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown ran in 0.08 seconds with accuracy of 74.65%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>SdcaMaximumEntropyMulti=>Unknown ran in 0.07 seconds with accuracy of 55.29%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LbfgsLogisticRegressionOva=>Unknown ran in 0.11 seconds with accuracy of 77.53%
ConvertType=>FeaturizeText=>Concatenate=>Unknown=>LbfgsLogisticRegressionOva=>Unknown ran in 0.10 seconds with accuracy of 74.90%
ConvertType=>FeaturizeText=>Conc

Microsoft.ML.AutoML.ExperimentResult`1[Microsoft.ML.Data.MulticlassClassificationMetrics] RunDetails index value 0 Microsoft.ML.AutoML.RunDetail`1[Microsoft.ML.Data.MulticlassClassificationMetrics] ValidationMetrics Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 1.1520647996815911 LogLossReduction 0.14393564785032045 MacroAccuracy 0.7541235334713596 MicroAccuracy 0.762 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 1.1427545367264178, 1.16242499424014, 1.2166878715626754, 1.118421132898133 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.872093023255814, 0.7933884297520661, 0.8421052631578947, 0.6728110599078341 ] PerClassRecall [ 0.75, 0.7619047619047619, 0.7111111111111111, 0.7934782608695652 ] Counts index value 0 [ 75, 4, 0, 21 ] 1 [ 4, 96, 0, 26 ] 2 [ 0, 2, 64, 24 ] 3 [ 7, 19, 12, 146 ] NumberOfClasses 4 Model index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single Size 11774 RawType Microsoft.ML.Data.VBuffer`1[System.Single] Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer Exception <null> TrainerName ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown RuntimeInSeconds 1.369 Estimator Microsoft.ML.Data.EstimatorChain`1[Microsoft.ML.ITransformer] LastEstimator Microsoft.ML.Transforms.KeyToValueMappingEstimator 1 Microsoft.ML.AutoML.RunDetail`1[Microsoft.ML.Data.MulticlassClassificationMetrics] ValidationMetrics Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 0.615563041593169 LogLossReduction 0.5425938050061206 MacroAccuracy 0.7488423050379572 MicroAccuracy 0.77 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 0.5186542419226803, 0.584890148731861, 1.1713885131726622, 0.4173639765139265 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.9047619047619048, 0.7611940298507462, 0.8833333333333333, 0.6936936936936937 ] PerClassRecall [ 0.76, 0.8095238095238095, 0.5888888888888889, 0.8369565217391305 ] Counts index value 0 [ 76, 13, 0, 11 ] 1 [ 3, 102, 0, 21 ] 2 [ 0, 1, 53, 36 ] 3 [ 5, 18, 7, 154 ] NumberOfClasses 4 Model index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single Size 11774 RawType Microsoft.ML.Data.VBuffer`1[System.Single] Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer Exception <null> TrainerName ConvertType=>Featur

In [148]:
// Grab the trained model for later
ITransformer model = result.BestRun.Model;

model

index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single RawType System.Single Size 11774 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

### Model Evaluation

In [149]:
// Identify the best run's algorithm
result.BestRun.TrainerName

ConvertType=>FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown

In [150]:
// Take a look a the best run's metrics
result.BestRun.ValidationMetrics

Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 0.5298679895962074 LogLossReduction 0.6062711946724723 MacroAccuracy 0.7882263630089716 MicroAccuracy 0.802 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 0.4163109759501623, 0.5551020055146971, 0.9503009559124491, 0.3686573830223658 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.9438202247191011, 0.8095238095238095, 0.8805970149253731, 0.7155963302752294 ] PerClassRecall [ 0.84, 0.8095238095238095, 0.6555555555555556, 0.8478260869565217 ] Counts index value 0 [ 84, 5, 0, 11 ] 1 [ 4, 102, 0, 20 ] 2 [ 0, 0, 59, 31 ] 3 [ 1, 19, 8, 156 ] NumberOfClasses 4

In [151]:
result.BestRun.ValidationMetrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||================================
PREDICTED ||     E |    ET |     M |     T | Recall
TRUTH     ||================================
        E ||    84 |     5 |     0 |    11 | 0.8400
       ET ||     4 |   102 |     0 |    20 | 0.8095
        M ||     0 |     0 |    59 |    31 | 0.6556
        T ||     1 |    19 |     8 |   156 | 0.8478
          ||================================
Precision ||0.9438 |0.8095 |0.8806 |0.7156 |


### Generating Predictions

In [152]:
    /// <summary>
    /// This class represents a set of confidences in the various ESRB ratings for a single video game
    /// </summary>
    public class ESRBPrediction
    {
        [ColumnName("PredictedLabel")]
        public string ESRBRating { get; set; }

        [ColumnName("Score")]
        public float[] Score { get; set; }
    }

In [153]:
// Create a PredictionEngine that can generate predictions for games we've not seen before
PredictionEngine<GameInfo, ESRBPrediction> predictor =
    Context.Model.CreatePredictionEngine<GameInfo, ESRBPrediction>(
            transformer: model);

In [154]:
// Create a list of sample games
List<GameInfo> games = new() {
    new GameInfo()
    {
        Title = "Comic Doggo Side Scroller for Teens",
        CartoonViolence = true,
        MildLanguage = true,
        CrudeHumor = true,
        Violence = true,
    },
    new GameInfo()
    {
        Title = "Kinda Sus",
        MildCartoonViolence = true
    },
    new GameInfo()
    {
        Title = "The Earthlings are Coming",
        MildViolence = true,
        MildFantasyViolence = true,
    },
    new GameInfo()
    {
        Title = "Shoddy Surgeon Simulator",
        BloodAndGore = true,
        DrugReference = true,
        PartialNudity = true,
    },
    new GameInfo()
    {
        Title = "Assistant to the Lawn Service Manager 2023",
        MildLanguage = true,
        CrudeHumor = true,
        AlcoholReference = true,
    },
    new GameInfo()
    {
        Title = "Intense Shoot-o-rama: Why would anyone play this edition",
        BloodAndGore = true,
        DrugReference = true,
        AlcoholReference = true,
        Nudity = true,
        StrongLanguage = true,
        SexualContent = true,
        SexualThemes = true,
        MatureHumor = true,
        IntenseViolence = true,
        CrudeHumor = true,
    }
};

games

index value 0 {Comic Doggo Side Scroller for Teens} Title Comic Doggo Side Scroller for Teens Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence True CrudeHumor True DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage True MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence True ESRBRating <null> 1 {Kinda Sus} Title Kinda Sus Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor False DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence True MildFantasyViolence False MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 2 {The Earthlings are Coming} Title The Earthlings are Coming Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor False DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence True MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence True NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 3 {Shoddy Surgeon Simulator} Title Shoddy Surgeon Simulator Console False AlcoholReference False AnimatedBlood False Blood False BloodAndGore True CartoonViolence False CrudeHumor False DrugReference True FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage False MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity True SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 4 {Assistant to the Lawn Service Manager 2023} Title Assistant to the Lawn Service Manager 2023 Console False AlcoholReference True AnimatedBlood False Blood False BloodAndGore False CartoonViolence False CrudeHumor True DrugReference False FantasyViolence False IntenseViolence False Language False Lyrics False MatureHumor False MildBlood False MildCartoonViolence False MildFantasyViolence False MildLanguage True MildLyrics False MildSuggestiveThemes False MildViolence False NoDescriptors False Nudity False PartialNudity False SexualContent False SexualThemes False SimulatedGambling False StrongLanguage False StrongSexualContent False SuggestiveThemes False UseOfAlcohol False UseOfDrugsAndAlcohol False Violence False ESRBRating <null> 5 {Intense Shoot-o-rama: Why would anyone play this edition} Title Intense Shoot-o-rama: Why would anyone play this edition Console False AlcoholReference True AnimatedBlood False Blood False BloodAndGore True CartoonViolence False CrudeHumor True DrugReference True FantasyViolence False IntenseViolence True Language False Lyrics False MatureHumor True MildBlood False MildCartoonViolence False MildFantasyVio

In [155]:
foreach (GameInfo game in games) {
    ESRBPrediction prediction = predictor.Predict(game);

    Console.WriteLine($"Predicted {prediction.ESRBRating} for {game.Title}");

    Console.Write("   Probabilities (E/ET/M/T): ");
    foreach (float probability in prediction.Score) {
        Console.Write($"{probability:P} ");
    }
    Console.WriteLine();
    Console.WriteLine();
}

Predicted T for Comic Doggo Side Scroller for Teens
   Probabilities (E/ET/M/T): 3.14% 44.69% 0.38% 51.79% 

Predicted E for Kinda Sus
   Probabilities (E/ET/M/T): 80.77% 12.74% 0.77% 5.72% 

Predicted E for The Earthlings are Coming
   Probabilities (E/ET/M/T): 61.23% 1.77% 0.78% 36.22% 

Predicted M for Shoddy Surgeon Simulator
   Probabilities (E/ET/M/T): 0.32% 0.95% 84.17% 14.56% 

Predicted ET for Assistant to the Lawn Service Manager 2023
   Probabilities (E/ET/M/T): 33.66% 44.92% 2.88% 18.54% 

Predicted M for Intense Shoot-o-rama: Why would anyone play this edition
   Probabilities (E/ET/M/T): 0.02% 22.27% 69.83% 7.88% 



### Save the Trained Model for Later

Because training models takes time, we can save the resulting model to a Zip file. This file can then be deployed elsewhere.

In [156]:
Context.Model.Save(model, inputSchema:null, "Model.zip");

Console.WriteLine("Model saved");

Model saved


### Load the Model

Most of the time you just want to use a trained model, not retrain it every time. This is how you'd do that.

In [157]:
ITransformer loadedModel = Context.Model.Load("Model.zip", out _);

loadedModel // After this, we'd build a PredictionEngine

index type value 0 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 1 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 2 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 3 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 4 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName __Features__ FeatureColumnType Vector Dimensions [ 11774 ] IsKnownSize True ItemType Single RawType System.Single Size 11774 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.OneVersusAllModelParameters 5 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

---

**Note:** models can also be saved to / loaded from ONNX format and used in other machine learning platforms and frameworks.